In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import time
from scipy.ndimage import zoom
%matplotlib inline

In [ ]:
def create_deck(deck_size, p_lotus):
    deck = np.random.uniform(size=deck_size)
    deck = 1 + np.floor(deck-p_lotus)
    return deck

def draw(hand, deck, n):
    hand = np.append(hand,deck[range(n)])
    deck = np.delete(deck,range(n))
    return (hand, deck)

def play_lotuses(hand, n_loti):
    ind = np.where(hand == 0)[0]
    n_loti += len(ind)
    hand = np.delete(hand,ind)
    return (hand, n_loti)

def can_play_wheel(hand, n_loti):
    ind = np.where(hand == 1)[0]
    if len(ind) > 0 and n_loti > 0:
        return True
    else:
        return False
    
def play_wheel(hand, deck, n_loti):
    n_loti -= 1
    hand = np.empty(0)
    (hand,deck) = draw(hand, deck, 7)
    return (hand, deck, n_loti)

def opponent_has_lost(deck, deck_size):
    if deck_size - len(deck) > 40:
        return True
    else:
        return False

def play_games(n_iter, p_lotus, deck_size, is_on_the_draw):
    winning_turn = np.zeros(n_iter)
    for i in range(n_iter):
        n_lotuses_in_play = 0
        n_turns = 0
        game_ended = False

        deck = create_deck(deck_size,p_lotus)
        hand = np.empty(0)
        #starting hand
        (hand, deck) = draw(hand,deck,7)
        # mulligan logic would go here if it was allowed

        #start taking turns
        while not game_ended:
            n_turns += 1
            #if is_on_the_draw or n_turns > 1:
            (hand, deck) = draw(hand,deck,1)
            if opponent_has_lost(deck, deck_size):
                winning_turn[i] = n_turns
                game_ended = True
            elif (len(deck)<7):
                winning_turn[i] = -n_turns
                game_ended = True

            (hand, n_lotuses_in_play) = play_lotuses(hand, n_lotuses_in_play)
            while can_play_wheel(hand, n_lotuses_in_play) and not game_ended:
                (hand, deck, n_lotuses_in_play) = play_wheel(hand, deck, n_lotuses_in_play)
                (hand, n_lotuses_in_play) = play_lotuses(hand, n_lotuses_in_play)
                
                if opponent_has_lost(deck, deck_size):
                    winning_turn[i] = n_turns
                    game_ended = True
                elif (len(deck)<7):
                    winning_turn[i] = -n_turns
                    game_ended = True
    return winning_turn



In [ ]:
n_iter = 10000
deck_size = 50
is_on_the_draw = False
p_lotus = np.linspace(0.0, 1.0, deck_size + 1)
average_winning_turn = np.zeros_like(p_lotus)
p_win_by_turn_1 = np.zeros_like(p_lotus)
p_win_by_turn_2 = np.zeros_like(p_lotus)
p_win_by_turn_3 = np.zeros_like(p_lotus)
p_draw = np.zeros_like(p_lotus)

start_time = time.time()

for i in range(len(p_lotus)):
    p = p_lotus[i]
    winning_turns = play_games(n_iter, p, deck_size, is_on_the_draw)
    
    p_win_by_turn_1[i] = float(len(np.where((winning_turns > 0) & (winning_turns <= 1))[0]))/float(n_iter)
    p_win_by_turn_2[i] = float(len(np.where((winning_turns > 0) & (winning_turns <= 2))[0]))/float(n_iter)
    p_win_by_turn_3[i] = float(len(np.where((winning_turns > 0) & (winning_turns <= 3))[0]))/float(n_iter)
    p_draw[i] = float(len(np.where(winning_turns < 0)[0]))/float(n_iter)
    average_winning_turn[i] = np.average(winning_turns[winning_turns > 0])
    print('Simulated ' + str(i+1) + ' / ' + str(len(p_lotus)) + ' cases in '+ str(time.time()-start_time) + ' seconds.')

print('Simulation took ' + str(time.time()-start_time) + ' seconds.')

In [ ]:
font = {'family' : 'sans-serif', 'weight' : 'normal', 'size' : 22}
plt.rc('font', **font)
n_lotus = (np.linspace(0, deck_size, deck_size + 1)).astype(int)

plt.plot(p_lotus,average_winning_turn,'o-')
plt.gca().set_ylim(0,5)
plt.grid()
plt.gca().set_xlabel('Fraction of Black Lotuses in deck')
plt.gca().set_ylabel('Average winning turn')
plt.gcf().set_size_inches(10,6)
plt.savefig('deck_size_'+str(deck_size)+'_'+'ave_win_turn.jpg',dpi=100)
plt.show()


plt.plot(p_lotus,p_win_by_turn_1,'s-', label = 'Win on turn 1')
plt.plot(p_lotus,p_win_by_turn_2,'o-', label = 'Win on turn 2')
plt.plot(p_lotus,p_win_by_turn_3,'.-', label = 'Win on turn 3')
#plt.plot(p_lotus,p_draw,'r--', label = 'Draw')
#plt.gca().set_ylim(0,5)
plt.grid()
plt.gca().set_xlabel('Fraction of Black Lotuses in deck')
plt.gca().set_ylabel('Probability')
plt.gcf().set_size_inches(10,6)
plt.legend()
plt.savefig('deck_size_'+str(deck_size)+'_'+'turn3_win_prob.jpg',dpi=100)
plt.show()

plt.plot(n_lotus,p_win_by_turn_1,'s-', label = 'Win on turn 1')
plt.plot(n_lotus,p_win_by_turn_2,'o-', label = 'Win on turn 2')
plt.plot(n_lotus,p_win_by_turn_3,'.-', label = 'Win on turn 3')
#plt.plot(p_lotus,p_draw,'r--', label = 'Draw')
#plt.gca().set_ylim(0,5)
plt.grid()
plt.gca().set_xlabel('Number of Black Lotuses in deck')
plt.gca().set_ylabel('Probability')
plt.gcf().set_size_inches(10,6)
plt.legend()
plt.savefig('deck_size_'+str(deck_size)+'_'+'turn3_win_prob_number_of_BL.jpg',dpi=100)
plt.show()

plt.plot(p_lotus,p_win_by_turn_1,'s-', label = 'Win on turn 1')
plt.plot(p_lotus,p_win_by_turn_2,'o-', label = 'Win on turn 2')
plt.plot(p_lotus,p_win_by_turn_3,'.-', label = 'Win on turn 3')
plt.plot(p_lotus,p_draw,'r--', label = 'Draw')
#plt.gca().set_ylim(0,5)
plt.grid()
plt.gca().set_xlabel('Fraction of Black Lotuses in deck')
plt.gca().set_ylabel('Probability')
plt.gcf().set_size_inches(10,6)
plt.legend()
plt.savefig('deck_size_'+str(deck_size)+'_'+'turn3_win_and_draw_prob.jpg',dpi=100)
plt.show()


In [ ]:
n_iter = 100000
deck_size = np.linspace(43,52,52-43+1)
is_on_the_draw = False
n_lotus = np.linspace(15,25,25-15+1)
average_winning_turn = np.zeros((len(n_lotus),len(deck_size)))
p_win_by_turn_1 = np.zeros((len(n_lotus),len(deck_size)))
p_win_by_turn_2 = np.zeros((len(n_lotus),len(deck_size)))
p_win_by_turn_3 = np.zeros((len(n_lotus),len(deck_size)))

start_time = time.time()
nnn = 0
for i in range(len(n_lotus)):
    for j in range(len(deck_size)):
        p = n_lotus[i]/deck_size[j]
        ds = int(deck_size[j])
        winning_turns = play_games(n_iter, p, ds, is_on_the_draw)

        p_win_by_turn_1[i][j] = float(len(np.where((winning_turns > 0) & (winning_turns <= 1))[0]))/float(n_iter)
        p_win_by_turn_2[i][j] = float(len(np.where((winning_turns > 0) & (winning_turns <= 2))[0]))/float(n_iter)
        p_win_by_turn_3[i][j] = float(len(np.where((winning_turns > 0) & (winning_turns <= 3))[0]))/float(n_iter)
        average_winning_turn[i][j] = np.average(winning_turns[winning_turns > 0])
        nnn += 1
        print('Simulated ' + str(nnn) + ' / ' + str(len(n_lotus)*len(deck_size)) + ' cases in '+ str(time.time()-start_time) + ' seconds.')

print('Simulation took ' + str(time.time()-start_time) + ' seconds.')

In [ ]:
data = p_win_by_turn_3
llx = 15; ulx = 25; lly = 43; uly = 52;
datamax = np.amax(data)
datamaxind = np.unravel_index(np.argmax(data),np.shape(data))
max_x = llx + datamaxind[0] + 0.5
max_y = lly + datamaxind[1] + 0.5
plt.imshow(np.transpose(data), extent=[15,25,43,52], origin='lower', interpolation='none')
plt.xlabel('Number of Black Lotuses')
plt.ylabel('Deck size')
plt.colorbar()
plt.plot(max_x, max_y, 'rx')
plt.show()

In [ ]:
data = p_win_by_turn_1
llx = 15; ulx = 25; lly = 43; uly = 52;

zoomf = 10
smdata = zoom(data, zoomf, order=3)
X,Y = np.meshgrid(np.linspace(n_lotus[0], n_lotus[-1], len(n_lotus)*zoomf), np.linspace(deck_size[0], deck_size[-1], len(deck_size)*zoomf))
levels = np.asarray([0.90, 0.92, 0.94, 0.96, 0.97, 0.974, 0.99])
contours = plt.contour(X, Y, np.transpose(smdata), levels, colors='black')
plt.clabel(contours, inline=True, fontsize=8)
plt.imshow(np.transpose(smdata), extent=[15,25,43,52], origin='lower', cmap='viridis', interpolation='none')
plt.xlabel('Number of Black Lotuses',fontsize=20)
plt.ylabel('Deck size',fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

cb = plt.colorbar()
cb.ax.set_yticklabels(cb.ax.get_yticklabels(), fontsize=16)
cb.set_label('Turn 1 win probability',size=16)

plt.savefig('turn1_probability_map.jpg',dpi=300, bbox_inches='tight')
plt.show()



In [ ]:
for ds in range(len(deck_size)):
    plt.plot(n_lotus/deck_size[ds], p_win_by_turn_1[:,ds],'*',markersize = 6,label='Deck size = ' + str(int(deck_size[0]+ds)))


#plt.gcf().set_size_inches(15,10)
plt.legend()
plt.minorticks_on()
plt.grid(which = 'major', linestyle = '-')
#plt.grid(which = 'minor', linestyle = '--')
plt.xlabel('Fraction of Black Lotuses',fontsize=14)
plt.ylabel('Probability to win on turn 1',fontsize=14)
plt.savefig('probability_collapse.jpg',dpi=300, bbox_inches='tight')
plt.show()
